
$$
\begin{aligned}
(\frac{\phi^{n+1}-\phi^n}{\tau},v) + (M\nabla \mu^{n+1}, \nabla v) &= 0\ \ \ \ \forall v \in \mathbb{V} \\
(\mu^{n+1},w)-(\epsilon \nabla \phi^{n+1}, \nabla w) 
&= (\gamma ((\phi^n)^3 -\phi^n),w) \ \ \ \  \forall w \in \mathbb{V} 
\end{aligned}
$$


In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
import random
import numpy as np
# Defining parameters
epsilon = 0.02
gamma = 50
M = 1e-3
maxh = 0.01 
order = 1 
tau = 0.1            
T = 1
#Defining mesh and Finite element space
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
V = H1(mesh, order=1) 
fes = V*V 

#Defining trial and test functions
phi,mu = fes.TrialFunction()  #n+1
v,w = fes.TestFunction()

#Initialising phi_0 random values between -1 and 1
phi_old = GridFunction(V)
#random_values = np.random.uniform(-1, 1, V.ndof)
random_values= np.zeros(V.ndof)
phi_old.vec.data = random_values

In [2]:
def phi_exact(t):
    return t*cos(np.pi*x)*cos(np.pi*y)

def phi_dt(t):
    return cos(np.pi*x)*cos(np.pi*y)

def phi_lap(t):
    return -2*np.pi**2*phi_exact(t)

def mu_exact():
    return cos(np.pi*x)*cos(np.pi*y)

def mu_lap():
    return -2*np.pi**2*mu_exact()

In [3]:
def FullyExplicitScheme(mesh,fes,phi,mu,v,w,phi_old,epsilon,gamma,M,tau,T):

    #Defining Bilinear form
    A = BilinearForm(fes)
    A += phi*v*dx+tau*M*grad(mu)*grad(v)*dx + mu*w*dx-epsilon*grad(phi)*grad(w)*dx
    A.Assemble()

    
    gfu = GridFunction(fes)
    gfut = GridFunction(gfu.space, multidim = 0)
    gfut.AddMultiDimComponent(gfu.vec)

    t=0
    while t<T:
        t+=tau
        rhs1 = phi_dt(t)-M*mu_lap() 
        rhs2 = mu_exact() - gamma*(phi_exact(t)**3 - phi_exact(t)) + epsilon*phi_lap(t)
            
        #Defining linear form
        f=LinearForm(fes)
        f+= tau*rhs1*v*dx+ phi_old*v*dx+gamma*(phi_old**3-phi_old)*w*dx + rhs2*w*dx
        f.Assemble()

        #Solving for phi
        gfu.vec.data = A.mat.Inverse(freedofs = fes.FreeDofs())*f.vec

        e_h = gfu.components[0]-phi_exact(t)
        norm = np.sqrt(Integrate(e_h*e_h,mesh))
        print(norm)

        #Storing and updating phi
        gfut.AddMultiDimComponent(gfu.vec)
        phi_old.vec.data = gfu.components[0].vec

    return gfut,gfu

In [4]:
gfut,gfu = FullyExplicitScheme(mesh,fes,phi,mu,v,w,phi_old,epsilon,gamma,M,tau,T)
Draw(gfut.components[0], mesh, interpolate_multidim = True, animate = True)

0.004904451142663343
0.010132265282804637
0.01563009737241884
0.02167849872382366
0.02942487302916763
0.04131548124983854
0.06110574283147397
0.09522993746712079
0.15786775053845858
0.27926832940854124
0.49177862432515146


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [5]:
from ngsolve import *
from ngsolve.webgui import Draw

#Defining mesh and Finite element space
mesh = Mesh(unit_square.GenerateMesh(maxh=0.01))
V = H1(mesh, order=1) 


gfu = GridFunction(V)
gfut = GridFunction(gfu.space, multidim = 0)
gfut.AddMultiDimComponent(gfu.vec)

#scene = Draw(exact,mesh)
t=0
while t<T:
    t += tau
    exact = t*cos(np.pi*x)*cos(np.pi*y)
    gfu.Set(exact)     
    gfut.AddMultiDimComponent(gfu.vec)
    #scene.Redraw(exact,mesh)
Draw(gfut, mesh, interpolate_multidim = True, animate = True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene